In [2]:
import keras
import itertools
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.generic_utils import CustomObjectScope
from keras.layers import DepthwiseConv2D, ReLU
from sklearn.metrics import confusion_matrix

In [4]:
model_directory ='../models/mobileNet/mobileNet_mobile_net.35-0.72.hdf5'

with CustomObjectScope({'ReLU': ReLU, 'DepthwiseConv2D': DepthwiseConv2D}):
    model = keras.models.load_model(model_directory)

In [5]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [6]:
batch_size = 128
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_generator = datagen.flow_from_directory(
        '../data/external/compcars_data/train_image',
        target_size=(224, 224),
        batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
        '../data/external/compcars_data/test_image',
        target_size=(224, 224),
        batch_size=batch_size)

generators = {'train': train_generator, 'validation': validation_generator}

Found 79528 images belonging to 163 classes.
Found 57114 images belonging to 163 classes.


In [7]:
[{k:model.evaluate_generator(v, steps=v.samples/batch_size)} for k,v in generators.items()]

KeyboardInterrupt: 

In [10]:
i = 'validation'
labels = []
predictions = []
g = generators[i]
for i in range(1):
    app_predictions = model.predict(g[i][0])
    app_predictions = [np.argmax(pred) for pred in app_predictions]
    a = [np.argmax(bleche) for bleche in g[i][1]]
    labels.extend(a)
    predictions.extend(app_predictions)
    
cm = confusion_matrix(labels, predictions)

In [ ]:
plot_confusion_matrix(cm, g.class_indices.keys(), normalize=True)